In [1]:
!nvidia-smi

Tue Jun  1 17:26:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN V             Off  | 00000000:03:00.0 Off |                  N/A |
| 28%   41C    P8    24W / 250W |   9057MiB / 12066MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:83:00.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |     10MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

First, we grab matplotlib, and set the old "classic" style for some reason only Rui knows.

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('classic')

In [3]:
import numpy as np
import torch

These are the local imports. Make sure you import the correct model!

In [4]:
from model.models import AltCNN4Layer_D35_sp as Model
from model.collectdata import collect_data, collect_truth
from model.training import select_gpu
from model.plots import plot_ruiplot
from model.efficiency import pv_locations, efficiency
from model.core import modernize_state

Select a GPU here. Same numbering as the NVidia-SMI tool.

In [5]:
device = select_gpu(2)

1 available GPUs (initially using device 0):
  0 TITAN V


Pick a file to load.

In [6]:
validation = collect_data('data/Oct03_20K_val.h5',
                          batch_size=1,
                          slice=slice(100),
                          masking=True,
                          device=device)

Loading data...
Loaded data/Oct03_20K_val.h5 in 3.666 s
Constructing 100 event dataset took 5.047 s


In [7]:
import h5py
XY_file = 'data/Oct03_20K_val.h5'

with h5py.File(XY_file, mode='r') as XY:
    xmax = np.asarray(XY['Xmax'])
    ymax = np.asarray(XY['Ymax'])

> Note: to get the real PV locations, use `collect_truth('file.h5', pvs=True)` to collect PVs (or SVs with `pvs=False`)

Let's just see how many NaNs we have in the dataset.

In [8]:
print(*np.sum(np.isnan(validation.dataset.tensors[1].cpu().numpy()), 1))

16 6 0 12 18 28 29 12 23 16 17 79 20 18 37 0 12 43 11 21 38 0 35 23 22 22 23 33 27 16 0 23 17 56 17 38 16 45 12 12 12 22 22 11 12 44 17 45 6 17 39 28 33 17 39 62 53 29 6 6 5 46 53 34 22 29 17 63 17 12 18 22 50 18 28 12 41 29 22 34 35 17 42 11 57 27 16 22 23 33 23 33 16 45 30 0 6 16 28 17


In [9]:
PV = collect_truth('data/Oct03_20K_val.h5', pvs=True)
print('PV.n.shape =    ',  PV.n.shape)
print('PV.n[0].shape = ', *PV.n[0].shape)
print('PV.x[0] =       ', *PV.x[0])
print('PV.y[0] =       ', *PV.y[0])
print('PV.z[0] =       ', *PV.z[0])
print('PV.n[0] =       ', *PV.n[0])
print('PV.cat[0] =     ', *PV.cat[0])

Loaded data/Oct03_20K_val.h5 in 0.03177 s
PV.n.shape =     (20000,)
PV.n[0].shape =  7
PV.x[0] =        0.0067699254 -0.05386329 0.0018882935 -0.035012268 -0.010384369 0.045956675 -0.073287025
PV.y[0] =        0.00076032506 -0.001020242 -0.03236149 0.043514606 -0.08862019 -0.06502226 -0.0052683405
PV.z[0] =        163.4873 80.101074 1.0544614 -36.59213 115.55979 48.95428 126.700806
PV.n[0] =        34 10 4 23 4 29 4
PV.cat[0] =      1 1 0 1 0 1 0


In [10]:
import h5py
try:
    import awkward0 as awkward
except ModuleNotFoundError:
    import awkward

In [ ]:
x_list = []
y_list = []
z_list = []
n_list = []
c_list = []

p = 'p'

with h5py.File('temp.h5', mode='r') as XY:
    afile = awkward.persist.hdf5(XY)
    x_list.append(afile[f"{p}v_loc_x"])
    y_list.append(afile[f"{p}v_loc_y"])
    z_list.append(afile[f"{p}v_loc"])
    n_list.append(afile[f"{p}v_ntracks"])
    c_list.append(afile[f"{p}v_cat"])

print(x_list[0].JaggedArray)

In [ ]:
simple = awkward.JaggedArray.fromcounts([2,3], [1.,2,3,4,5])

with h5py.File('example.h5', mode='w') as f:
    af = awkward.persist.hdf5(f)
    af['example'] = simple
    
with h5py.File('example.h5', mode='r') as f:
    af = awkward.persist.hdf5(f)
    example = af['example']
    
print(example.JaggedArray)

In [ ]:
with h5py.File('temp.h5', mode='r') as XY:
    schema = XY['pv_loc_x']['schema.json'].value

In [ ]:
print(np.string_(schema))

In [ ]:
PV.n.flatten().max()

In [ ]:
PV.x

In [ ]:
PV.cat[0]

In [ ]:
SV = collect_truth('data/Oct03_20K_val.h5', pvs=False)
print('SV.n.shape =    ', SV.n.shape)
print('SV.n[0].shape = ', *SV.n[0].shape)
print('SV.x[0] =       ', *SV.x[0])
print('SV.y[0] =       ', *SV.y[0])
print('SV.z[0] =       ', *SV.z[0])
print('SV.n[0] =       ', *SV.n[0])
print('SV.cat[0] =     ', *SV.cat[0])

In [ ]:
model = Model().to(device)

Select a model to load. Make sure it matches the model you imported above.

> #### Mike note:
>
> If you use an old-style model, comment out the `d = modernize(d, 3)` line - that converts the old model key names to the new format.

In [ ]:
state = torch.load(
    '/share/lazy/schreihf/PvFinder/models/'
    '07Jan19_AltCNN4Layer_D35_sp_300epochs'
    '_240K_lr_3em5_bs256_Alt_Loss_A_5p5/'
    '07Jan19_AltCNN4Layer_D35_sp_300epochs'
    '_240K_lr_3em5_bs256_Alt_Loss_A_5p5_199.pyt'
)

In [ ]:
del state['fc1.weight'], state['fc1.bias']

In [ ]:
state = modernize_state(model, state)

In [ ]:

model.load_state_dict(state)
model.eval()

Let's grab the outputs and labels as normal numpy arrays.

In [ ]:
%%time
with torch.no_grad():
    outputs = model(validation.dataset.tensors[0]).cpu().numpy()
    labels = validation.dataset.tensors[1].cpu().numpy()

In [ ]:
fontdict = {
    'font.size':18,
    'font.weight':'bold'
}

And here's Rui's plotting code.

In [ ]:
inputs = validation.dataset.tensors[0].cpu().numpy().squeeze()
zvals = np.linspace(-100, 300, 4000, endpoint=False) + 0.05
finalmsg = ''
internal_count = 0
output_filename = '07Jan19_AltCNN4Layer_D35_sp_{number:02}.pdf'
#  None # Or set: '120000_3layer_{number:02}.pdf'

for event in range(2):
    input = inputs[event]
    label = labels[event]
    output = outputs[event]
    
    # Consistent parameters for the calls below
    parameters = {
        "threshold": 1e-2,
        "integral_threshold": .2,
        "min_width": 3
    }
    
    # Compute the "actual" efficenies and things
    ftruth = pv_locations(label, **parameters)
    fcomputed = pv_locations(output, **parameters)
    results = efficiency(label, output, difference=5.0, **parameters)
    
    # Add a line to the final results string (print at end)
    finalmsg += f"Event {event}: {results}\n"
    
    # Make sure bin numbers are integers
    truth = np.around(ftruth).astype(np.int32)
    computed = np.around(fcomputed).astype(np.int32)
    
    # Join arrays and remove any points closer than 5 bins
    # We plot over these "points of interest"
    poi = np.sort(np.concatenate([truth, computed]))
    poi = poi[np.concatenate([[True], np.fabs(np.diff(poi)) > 5])]
    
    print(f"\nEvent {event}:", results)
    
    for index, i in enumerate(poi):
        # Convert to location in z
        center = (i / 10) - 100
        
        # Collect items less than 5 apart as "true"
        b_truth = np.fabs(ftruth - i) <= 5
        b_comp = np.fabs(fcomputed - i) <= 5
        in_truth = np.any(b_truth)
        in_comp = np.any(b_comp)
        
        if in_truth and in_comp:
            msg = 'PV found'
        elif in_truth:
            msg = 'PV not found'
        elif np.any(np.isnan(label[i-3:i+3])):
            msg = 'Masked'
        else:
            msg = 'False positive'
            
        with plt.style.context(fontdict):
        
            fig, axs = plt.subplots(2, figsize=(12,10), sharex=True,
                                    gridspec_kw={'height_ratios':[2,1],
                                                'hspace':0.1})
        
            # ax1 is the axis that is tied to left (density)
            # ax2 is the axis that is tied to the right (probability)
            ax1, ax2 = plot_ruiplot(zvals, i, input, label, output, ax=axs[0])
            ax1.set_title(f"Event {event} @ {center:.1f} mm: {msg}",
                          fontdict=fontdict)


            msg = ""
            
            truth_centroid = (ftruth[b_truth] / 10) - 100
            for value in truth_centroid:
                msg += f"True: {value:.3f} mm\n"
                
            comp_centroid = (fcomputed[b_comp] / 10) - 100
            for value in comp_centroid:
                msg += f"Pred: {value:.3f} mm\n"
                
            if len(truth_centroid) == 1 and len(comp_centroid) == 1:
                diff = (comp_centroid[0] - truth_centroid[0]) * 1_000
                msg += f"∆: {diff:.0f} µm\n"
            
            ax1.text(.02, .8, msg,
                     transform=ax1.transAxes,
                     verticalalignment='top')
            
            print(f"\nEvent {event}.{index}:")
            
            # Plot and print PVs
            ax2.scatter(PV.z[event], np.ones_like(PV.z[event])*.4, s=50, color='C0')
            for x,y,z,n,cat in zip(PV.x[event], PV.y[event], PV.z[event], PV.n[event], PV.cat[event]):
                # Only print out if z in plotting range
                if center - 2.5 < z < center + 2.5:
                    print()
                    print(f'PV: {n} tracks (type {cat})')
                    print(f'  x: {x*1000:5.0f} μm')
                    print(f'  y: {y*1000:5.0f} μm')
                    print(f'  z: {z:8.3f} mm')
                    

            # Plot and print SVs
            ax2.scatter(SV.z[event], np.ones_like(SV.z[event])*.6, s=50, color='C1')
            for x,y,z,n,cat in zip(SV.x[event], SV.y[event], SV.z[event], SV.n[event], SV.cat[event]):
                # Only print out if z in plotting range
                if center - 2.5 < z < center + 2.5:
                    print()
                    print(f'SV: {n} tracks (type {cat})')
                    print(f'  x: {x*1000:5.0f} μm')
                    print(f'  y: {y*1000:5.0f} μm')
                    print(f'  z: {z:8.3f} mm')
            
            ax = axs[1]
            ax.plot((np.arange(4000) / 10) - 100, xmax[event]*1000000, label="x")
            ax.plot((np.arange(4000) / 10) - 100, ymax[event]*1000000, label="y")
            ax.set_xlim(ax1.get_xlim())
            ax.set_ylim(-150,150)
            ax.grid(axis='y')
            ax.set_ylabel('xy maximum [μm]')
            ax.legend(loc='best')
            
            ax.set_xlabel(ax1.get_xlabel())
            ax1.set_xlabel("")
            
            if output_filename:
                print()
                print(output_filename.format(number=internal_count))
            
            # Save and show
            if output_filename:        
                plt.savefig(output_filename.format(number=internal_count), transparent=True)
            plt.show()
            
            internal_count += 1
            
print(finalmsg)